In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from glob import glob
import os, math, time
import sys
sys.path.insert(1, '/home/gusinskaia/psr_tools')

import numpy as np
import pickle

import astropy
from astropy import units as u, constants as const
from astropy.time import Time
from astropy.visualization import quantity_support

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib as mpl

from scipy.sparse.linalg import eigsh
from scipy.optimize import curve_fit
import scipy.linalg

import load_data as ld
import ds_psr as dsa
import fit_thth as fth
import models_thth as mth
import wsrt_fits as wf
import gbt_fits as gf
import ththmod as THTH

plt.rcParams['figure.dpi'] = 150

# All the data:

In [ ]:
gbt_dir='/mnt/scratch-lustre/gusinskaia/triple_system/2021_GBT_dss/'
names=sorted(glob(gbt_dir+'*'))


actual_names=[x.split('/')[-1] for x in names]
mjds=[x.split('_')[0] for x in actual_names]
path_name=names[0].split(actual_names[0])[0]

print (mjds)
print (path_name)

In [ ]:
spec_59297=gf.load_new_gbt(59297)#too short

In [ ]:
spec_59299=gf.load_new_gbt(59299)

In [ ]:
spec_59324=gf.load_new_gbt(59324)

In [ ]:
spec_59363=gf.load_new_gbt(59363)

In [ ]:
spec_59399=gf.load_new_gbt(59399)

In [ ]:
spec_59403=gf.load_new_gbt(59403)

In [ ]:
spec_59426=gf.load_new_gbt(59426)#too short

In [ ]:
spec_59434=gf.load_new_gbt(59434)#too short

In [ ]:
spec_59464=gf.load_new_gbt(59464)#too short

In [ ]:
spec_59496=gf.load_new_gbt(59496)

In [ ]:
spec_59498=gf.load_new_gbt(59498)

In [ ]:
spec_59518=gf.load_new_gbt(59518)

In [ ]:
spec_59545=gf.load_new_gbt(59545)

In [ ]:
spec_59574=gf.load_new_gbt(59574)

In [ ]:
spec_59647_fin=gf.load_new_gbt(59647, shrink=[1,1])
spec_59647_st=gf.load_new_gbt(59646, shrink=[1,1])

In [ ]:
spec_59647=spec_59647_st+spec_59647_fin
spec_59647=spec_59647.shrink([10,1])

In [ ]:
spec_59647_st.mjd

In [ ]:
all_specs=[spec_59297,spec_59299,spec_59324,spec_59363,spec_59399,spec_59403,spec_59426,spec_59434,spec_59464,
           spec_59496,spec_59498,spec_59518,spec_59545,spec_59574,spec_59647]
most_specs=[spec_59299,spec_59324,spec_59363,spec_59399,spec_59403,
           spec_59496,spec_59498,spec_59518,spec_59545,spec_59574,spec_59647]

In [ ]:
spec_59647.mjd.mjd.mean()

In [ ]:
def plot_one_ss(spec, fig, rect, tclr='k'):
    fig.add_axes(rect)
    frame1=plt.gca()
    spec.plot_ss(new_fig=False, tau_lim=[0.0,1.25], vmin=1e3,vmax=5e6, cb=False)
    plt.title(spec.mjd.iso[int(len(spec.mjd.mjd)/2)][:10], color=tclr)

In [ ]:
t_clrs=['r','k','k','k','k','k','r','r','r','k','k','k','k','k','k',]
fig=plt.figure(figsize=(10,1.7), dpi=150)
for i in range(0,len(all_specs)):
    plot_one_ss(all_specs[i], fig, rect=[0.225*(i%5), -1.5*(i//5), 0.15, 1], tclr=t_clrs[i])
plt.show()

In [ ]:
fig=plt.figure(figsize=(10,1.7), dpi=150)
for i in range(0,len(most_specs)):
    plot_one_ss(most_specs[i], fig, rect=[0.225*(i%4), -1.5*(i//4), 0.15, 1])
plt.show()

In [ ]:
def plot_gbt_dsss(spec, save_it=False, fd_lim=[-1.5,1.5], ds=True, new_fig=True, fig=None, ss_xst=0.3):
    if new_fig is True:
        fig=plt.figure(figsize=(7,7), dpi=150)
    if ds is True:
        fig.add_axes([0.0,0.0,0.2,1])
        frame1=plt.gca()
        spec.select(freq_sel=[1315*u.MHz,1840*u.MHz]).plot_ds(new_fig=False)
        plt.title('DS')
    fig.add_axes([ss_xst,0.75,0.25,0.25])
    frame1=plt.gca()
    spec.plot_ss(new_fig=False,tau_lim=[0.0,1.22], fd_lim=fd_lim)
    plt.title('full SS\n %.1f - %.1f MHz'%(spec.f[0].value, spec.f[-1].value))
    fig.add_axes([ss_xst,0.0,0.25,0.25])
    frame1=plt.gca()
    spec.select(freq_sel=[1315*u.MHz,1540*u.MHz]).plot_ss(new_fig=False,tau_lim=[0.0,1.22], fd_lim=fd_lim)
    plt.title('Bottom band SS\n 1315 - 1540 MHz')
    fig.add_axes([ss_xst,0.375,0.25,0.25])
    frame1=plt.gca()
    spec.select(freq_sel=[1540*u.MHz,1840*u.MHz]).plot_ss(new_fig=False,tau_lim=[0.0,1.22], fd_lim=fd_lim)
    plt.title('Top band SS\n 1540 - 1840 MHz')
    if save_it is True:
        mjd_name=int(np.mean(spec.mjd.mjd))
        plt.savefig('GBT_%s_ds_ss.png'%mjd_name, format='png',bbox_inches='tight',dpi=150)

In [ ]:
plot_gbt_dsss(spec_59574)

In [ ]:
plot_gbt_dsss(spec_59574, ds=False)

In [ ]:
sp_59647=spec_59647.shrink([2,1])
sp_59647.I.shape

In [ ]:
plot_gbt_dsss(spec_59647, fd_lim=[-0.75,0.75])

In [ ]:
dur=(spec_59299.mjd[-1]-spec_59299.mjd[0]).to(u.hr).value
dur

In [ ]:
0.2*dur/dur_299

In [ ]:
fig=plt.figure(figsize=(7,6), dpi=150)
dur_299=(spec_59299.mjd[-1]-spec_59299.mjd[0]).to(u.hr).value
fig.add_axes([0.0,0.0,0.2,1])
frame1=plt.gca()
spec_59299.select(freq_sel=[1315*u.MHz,1840*u.MHz]).plot_ds(new_fig=False)
plt.title('MJD: 59299')
y_st=0.2+0.02

y_wid=((spec_59403.mjd[-1]-spec_59403.mjd[0]).to(u.hr).value)*0.2/dur_299
fig.add_axes([y_st,0.0,y_wid,1])
frame1=plt.gca()
spec_59403.select(freq_sel=[1315*u.MHz,1840*u.MHz]).plot_ds(new_fig=False)
frame1.axes.get_yaxis().set_ticks([])
plt.ylabel('')
plt.title('MJD: 59403')
y_st=y_st+y_wid+0.02

y_wid=((spec_59498.mjd[-1]-spec_59498.mjd[0]).to(u.hr).value)*0.2/dur_299
fig.add_axes([y_st,0.0,y_wid,1])
frame1=plt.gca()
spec_59498.select(freq_sel=[1315*u.MHz,1840*u.MHz]).plot_ds(new_fig=False)
frame1.axes.get_yaxis().set_ticks([])
plt.ylabel('')
plt.title('MJD: 59498')
y_st=y_st+y_wid+0.02

y_wid=((spec_59574.mjd[-1]-spec_59574.mjd[0]).to(u.hr).value)*0.2/dur_299
fig.add_axes([y_st,0.0,y_wid,1])
frame1=plt.gca()
spec_59574.select(freq_sel=[1315*u.MHz,1840*u.MHz]).plot_ds(new_fig=False)
frame1.axes.get_yaxis().set_ticks([])
plt.ylabel('')
plt.title('MJD: 59575')
y_st=y_st+y_wid+0.02

y_wid=((spec_59647.mjd[-1]-spec_59647.mjd[0]).to(u.hr).value)*0.2/dur_299
fig.add_axes([y_st,0.0,y_wid,1])
frame1=plt.gca()
spec_59647.select(freq_sel=[1315*u.MHz,1840*u.MHz]).plot_ds(new_fig=False)
frame1.axes.get_yaxis().set_ticks([])
plt.ylabel('')
plt.title('MJD: 59647')
plt.savefig('GBT_long_ds_v2.pdf', format='pdf',bbox_inches='tight',dpi=150)

In [ ]:
fig=plt.figure(figsize=(7,7), dpi=150)
plot_gbt_dsss(spec_59299, ds=False, new_fig=False, fig=fig, ss_xst=0.0, fd_lim=[-1,1])

plot_gbt_dsss(spec_59403, ds=False, new_fig=False, fig=fig, ss_xst=0.35, fd_lim=[-1,1])

plot_gbt_dsss(spec_59498, ds=False, new_fig=False, fig=fig, ss_xst=0.7, fd_lim=[-1,1])

plot_gbt_dsss(spec_59574, ds=False, new_fig=False, fig=fig, ss_xst=1.05, fd_lim=[-1,1])

plot_gbt_dsss(spec_59647, ds=False, new_fig=False, fig=fig, ss_xst=1.4, fd_lim=[-1,1])

plt.savefig('GBT_long_ss.png', format='png',bbox_inches='tight',dpi=150)

In [ ]:
spec_59647

In [ ]:
for i in range(0,9):
    sel_spec=spec_59363.select(freq_sel=[1312*u.MHz+(59*i)*u.MHz, 1312*u.MHz+(59*(i+1))*u.MHz])
    sel_spec.plot_ds(figsize=(2,2))

# Do the fit

In [ ]:
res_324=gf.fit_new_gbt(spec_59324,ntime=1, nfreq=9, freq_start=1312*u.MHz, freq_step=59*u.MHz)

In [ ]:
res_574=gf.fit_new_gbt(spec_59574,ntime=4, nfreq=9, freq_start=1312*u.MHz, freq_step=59*u.MHz)

In [ ]:
res_647=gf.fit_new_gbt(spec_59647,ntime=3, nfreq=9, par_lims=[0.05,1],freq_start=1312*u.MHz, freq_step=59*u.MHz)
with open('res_coh_%.2f_%s.pickle'%(np.mean(spec_59647.mjd.mjd),spec_59647.tel), 'wb') as e:
    pickle.dump(res_647, e, pickle.HIGHEST_PROTOCOL)

In [ ]:
res_647_eigv=gf.fit_new_gbt(spec_59647,ntime=3, nfreq=9, par_lims=[0.05,1],freq_start=1312*u.MHz,
                            freq_step=59*u.MHz,thth_method='coherent', chi2_method='Eigen')
with open('res_coh_Eigv_%.2f_%s.pickle'%(np.mean(spec_59647.mjd.mjd),spec_59647.tel), 'wb') as e:
    pickle.dump(res_647_eigv, e, pickle.HIGHEST_PROTOCOL)

In [ ]:
res_647_inc=gf.fit_new_gbt(spec_59647,ntime=3, nfreq=9, par_lims=[0.05,1],freq_start=1312*u.MHz,
                            freq_step=59*u.MHz,thth_method='incoherent')
with open('res_inc_%.2f_%s.pickle'%(np.mean(spec_59647.mjd.mjd),spec_59647.tel), 'wb') as e:
    pickle.dump(res_647_inc, e, pickle.HIGHEST_PROTOCOL)

In [ ]:
most_specs=[spec_59299,spec_59324,spec_59363,spec_59399,spec_59403,
           spec_59496,spec_59498,spec_59518,spec_59545,spec_59574]

ntimes=[4,1,1,1,4,1,4,1,1,4]

In [ ]:
for i in range(0,len(most_specs)):
    this_sp=most_specs[i]
    this_res=gf.fit_new_gbt(this_sp,ntime=ntimes[i], nfreq=9, freq_start=1312*u.MHz, freq_step=59*u.MHz)
    with open('res_coh_%.2f_%s.pickle'%(np.mean(this_sp.mjd.mjd),this_sp.tel), 'wb') as e:
        pickle.dump(this_res, e, pickle.HIGHEST_PROTOCOL)

In [ ]:
for i in range(0,len(most_specs)):
    this_sp=most_specs[i]
    this_res=gf.fit_new_gbt(this_sp,ntime=ntimes[i], nfreq=9, freq_start=1312*u.MHz, freq_step=59*u.MHz,
                           thth_method='incoherent')
    with open('res_inc_%.2f_%s.pickle'%(np.mean(this_sp.mjd.mjd),this_sp.tel), 'wb') as e:
        pickle.dump(this_res, e, pickle.HIGHEST_PROTOCOL)

In [ ]:
for i in range(0,len(most_specs)):
    this_sp=most_specs[i]
    this_res=gf.fit_new_gbt(this_sp,ntime=ntimes[i], nfreq=9, freq_start=1312*u.MHz, freq_step=59*u.MHz,
                           thth_method='coherent', chi2_method='Eigen')
    with open('res_coh_Eigv_%.2f_%s.pickle'%(np.mean(this_sp.mjd.mjd),this_sp.tel), 'wb') as e:
        pickle.dump(this_res, e, pickle.HIGHEST_PROTOCOL)

# Save results

In [ ]:
#with open('res_coh_%.2f_%s.pickle'%(np.mean(spec_59574.mjd.mjd),spec_59574.tel), 'wb') as e:
#    pickle.dump(res_574, e, pickle.HIGHEST_PROTOCOL)

# Load pickles

In [ ]:
list_pickles=sorted(glob('*GBT*.pickle'))
list_pickles

In [ ]:
res_this=pickle.load(open(list_pickles[0], 'rb'))

In [ ]:
res_this

# Examine results

In [ ]:
def plot_veff_freq(dic):
    for i in range(0,len(dic['times'])):
        plt.errorbar(dic['freqs'],dic['dveffs'][i,:], yerr=dic['dveff_e'][i,:],ls='none',marker='o',
                     label='%.2f'%dic['times'][i])

    plt.legend()
    #plt.show()

     
def plot_chi2s(dic):
    for i in range(0,np.shape(dic['dveff_chi2'])[0]):
        for k in range(0,np.shape(dic['dveff_chi2'])[1]):
            plt.plot(dic['dveff_ar'][i,k,:],dic['dveff_chi2'][i,k,:], ls=':', lw=1,
                     label='%.2f'%dic['freqs'][k])
        plt.plot(dic['dveff_ar'][i,k,:],np.mean(dic['dveff_chi2'][i,:,:], axis=0), color='k', ls='-', lw=2,
                     label='mean')
        plt.title(dic['times'][i])
        plt.legend(loc=(1.01,0.2))
        plt.show()

        

In [ ]:
plot_veff_freq(res_this)
plt.ylim(0.0,2.5)

In [ ]:
plot_veff_freq(res_324)

In [ ]:
plot_chi2s(res_324)

# Apply masks

In [ ]:
res_this

In [ ]:
res_this['dveffs'].shape

In [ ]:
res_this['times']

In [ ]:
veffmask_coh_chi2=open('veff_masks_coh_chi2.txt')

In [ ]:
masks=veffmask_coh_chi2.readlines()

In [ ]:
masks